In [2]:
!git clone https://github.com/attardi/wikiextractor.git


Cloning into 'wikiextractor'...


In [3]:
!pip install wikiextractor


  Using cached wikiextractor-3.0.6-py3-none-any.whl (46 kB)


In [1]:
import string
import re
intab_l = "ạảãàáâậầấẩẫăắằặẳẵóòọõỏôộổỗồốơờớợởỡéèẻẹẽêếềệểễúùụủũưựữửừứíìịỉĩýỳỷỵỹđ"
ascii_lowercase = 'abcdefghijklmnopqrstuvwxyz'
ascii_uppercase = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'
digits = '0123456789'
punctuation = r"""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""
whitespace = ' '
accept_strings =  intab_l + ascii_uppercase + ascii_lowercase + digits + punctuation + whitespace
r = re.compile('^[' + accept_strings + ']+$')


In [2]:
def _check_tieng_viet(seq):
  if re.match(r, seq.lower()):
    return True
  else:
    return False

_check_tieng_viet('Duy Anh đẹp trai')

True

In [3]:
import pickle
import json
from tqdm import tqdm
import glob2
idx = 0

for path in tqdm(glob2.glob('output/*/*')):
  # Đọc nội dung của các văn bản từ folder output. Content sẽ chứa nhiều row, mỗi row là một json data
  with open(path, 'r', encoding='utf8') as f:
    content = f.readlines()
    for row in content:
      # Convert row sang json
      art_json = json.loads(row)
      # Lấy nội dung văn bản
      art_cont = art_json['text']
      art_cont = re.sub("(\s)+", r"\1", art_cont)
      # Chia văn bản thành các câu tại vị trí xuống dòng
      art_seqs = art_cont.split("\n")
      # Lưu các dòng là tiếng việt vào file 'train_tieng_viet.txt'.
      # Mỗi dòng có định dạng: index{10digits} sequence
      for seq in art_seqs:
                if _check_tieng_viet(seq):
                    with open('train_tieng_viet.txt', 'a') as f:
                        f.writelines([seq + '\n'])
                    idx += 1

100%|██████████| 1891/1891 [25:05<00:00,  1.26it/s]


In [19]:
with open('train_tieng_viet.txt', 'r', encoding='utf-8') as f:
  data_pro = f.readlines()

print('Number of sequences: ', len(data_pro))
print('First sequence: ', data_pro[1])

Number of sequences:  3478862
First sequence:  Internet Society nằm ở gần thủ đô Washington, DC, Hoa Kỳ và Geneva, Thụy Sĩ. Số hội viên của nó bao gồm hơn 145 tổ chức thành viên và hơn 65.000 cá nhân. Thành viên còn có thể tự lập một chi nhánh của tổ chức tùy theo vị trí hoặc sở thích. Hiện nay tổ chức có tới 90 chi nhánh trên toàn thế giới.



In [20]:
from tqdm import tqdm
for i in tqdm(range(100)):
  print(data_pro[i])


100%|██████████| 100/100 [00:00<00:00, 99983.41it/s]

Internet Society hay ISOC là một tổ chức quốc tế hoạt động phi lợi nhuận, phi chính phủ và bao gồm các thành viên có trình độ chuyên ngành. Tổ chức này chú trọng đến: tiêu chuẩn, giáo dục và các vấn đề về chính sách. Với trên 145 tổ chức thành viên và 65.000 thành viên cá nhân, ISOC bao gồm những con người cụ thể trong cộng đồng Internet. Mọi chi tiết có thể tìm thấy tại website của ISOC.

Internet Society nằm ở gần thủ đô Washington, DC, Hoa Kỳ và Geneva, Thụy Sĩ. Số hội viên của nó bao gồm hơn 145 tổ chức thành viên và hơn 65.000 cá nhân. Thành viên còn có thể tự lập một chi nhánh của tổ chức tùy theo vị trí hoặc sở thích. Hiện nay tổ chức có tới 90 chi nhánh trên toàn thế giới.

Nhiệm vụ và mục đích hoạt động.

Bảo đảm, cổ vũ cho sự phát triển, mở rộng và sử dụng Internet được thuận lợi nhất cho mọi người trên toàn thế giới.

Tiếng Việt, cũng gọi là tiếng Việt Nam hay Việt ngữ là ngôn ngữ của người Việt và là ngôn ngữ chính thức tại Việt Nam. Đây là tiếng mẹ đẻ của khoảng 85% dân cư

In [6]:
!pip install langdetect

     -------------------------------------- 981.5/981.5 kB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993253 sha256=ec7aca19e37659134448cd37aa1542d6c83bf5e5aa186f2220555738ec2cf5da
  Stored in directory: c:\users\duyan\appdata\local\pip\cache\wheels\c4\16\af\1889804d8b7c0c041cadee8e29673a938a332acbf2865c70a1
Successfully built langdetect


In [10]:
from tqdm import tqdm
import string

tu_ngoai_ngu = []

def contains_vietnamese(word):
    vietnamese_characters = set("àáảãạăắằẳẵặâấầẩẫậèéẻẽẹêếềểễệìíỉĩịòóỏõọôốồổỗộơớờởỡợùúủũụưứừửữựỳỵỷỹý")
    non_vietnamese_characters = set("wzj")
    return any(char in vietnamese_characters for char in word.lower()) or any(char in non_vietnamese_characters for char in word.lower()) or word.isupper()

def remove_punctuation(word):
    return ''.join(char for char in word if char not in string.punctuation)

def kiem_tra_so_am_tiet(tu):
    diphthongs = ["ai", "oi", "ay", "uy", "au","ao","oa", "gia", "eo", "ia", "ua", "ue", "iu", "io", "ua", "uo", "uô", "ươ", "ưa", "uơ", "ui","ia", "ie", "iu", "oi", "ôi", "ơi", "ua", "uô", "ươ", "uy", "uay", "uôy"]
    
    am_tiet = 0
    tu_lower = tu.lower()
    
    i = 0
    while i < len(tu_lower):
        if tu_lower[i:i+2] in diphthongs:
            am_tiet += 1
            i += 2
        elif tu_lower[i] in 'aeiouyáàạảãăắằặẳẵâấầậẩẫéèẹẻẽêếềệểễóòọỏõọôốồộổỗơớờợởỡíìịỉĩúùụủũưứừựửữýỳỵỷỹ':
            am_tiet += 1
            i += 1
        else:
            i += 1

    return am_tiet

def check_uppercase_count(word):
    uppercase_count = sum(1 for char in word if char.isupper())
    return uppercase_count

with open("test.txt", "r", encoding="utf-8") as file:
    data_pro = [line.replace('\n', '') for line in file]

# Build a list of foreign words
eng = []
for i in tqdm(range(len(data_pro))):
    # Handle hyphens by replacing with spaces
    cleaned_line = data_pro[i].replace('-', ' ')
    
    # Split the line into individual words
    word_list = cleaned_line.split(' ')
    
    for word in word_list:
        cleaned_word = remove_punctuation(word)

        if cleaned_word != '' and not contains_vietnamese(cleaned_word) and kiem_tra_so_am_tiet(cleaned_word) > 1 or any(char in 'wzj' for char in cleaned_word) or check_uppercase_count(cleaned_word) >= 2:
            tu_ngoai_ngu.append(cleaned_word)

# Save the list of filtered words to a separate file
with open("filtered_words.txt", "w", encoding="utf-8") as filter:
    for word in tu_ngoai_ngu:
        filter.write(word + '\n')


100%|██████████| 28/28 [00:00<00:00, 2113.84it/s]


In [11]:
# Chuyển tu_ngoai_ngu thành một set
tu_ngoai_ngu_set = set(tu_ngoai_ngu)

with open("test.txt", "w", encoding="utf-8") as data1:
    for i, line in tqdm(enumerate(data_pro), desc="Processing lines", unit="lines"):
        word_list = line.split(' ')
        sen_words = []

        for word in word_list:
            cleaned_word = remove_punctuation(word)
            if cleaned_word:
                original_word = word
                if cleaned_word in tu_ngoai_ngu_set:
                    index = tu_ngoai_ngu.index(cleaned_word)
                    sen_words.append("ENG_" + str(index))
                else:
                    sen_words.append(original_word)

        sen = ' '.join(sen_words)
        data1.write(sen + '\n')


Processing lines: 28lines [00:00, 9006.17lines/s]


In [15]:
import random
import string
from unidecode import unidecode
import numpy as np

acronym = {"không": ["ko", "k"], "anh": ["a"], "em": ["e"], "biết": ["bít", "bt"], "giờ": ["h"], "gì": ["j"], "muốn": ["mún"],
           "học": ["hc", "hok"], "làm": ["lm"], "yêu": ["iu"], "chồng": ["ck"], "vợ": ["vk"], "ông": ["ô"], "được": ["đc", "dc"],
           "tôi": ["t"], "bạn": ["b","bn", "pạn"], "nước": ["nc"], "không liên quan": ["klq"], "không hiểu sao": ["khs"], "chả hiểu sao": ["chs"],
           "rồi": ["òy", "oy", "r"], "vậy": ["v", "z"], "mới": ["ms"], "vẫn": ["vx"], "Không": ["Ko", "K"], "Anh": ["A"], "Em": ["E"],
           "Biết": ["Bít", "Bt"], "Giờ": ["H"], "Gì": ["J"], "Muốn": ["Mún"], "Học": ["Hc", "Hok"], "Làm": ["Lm"], "Yêu": ["Iu"], "Chồng": ["Ck"],
           "Vợ": ["Vk"], "Ông": ["Ô"], "Được": ["Đc", "Dc"], "Tôi": ["T"], "Bạn": ["B","Bn", "Pạn"], "Nước": ["Nc"], "Không Liên Quan": ["Klq"],
           "Không Hiểu Sao": ["Khs"], "Chả Hiểu Sao": ["Chs"], "Rồi": ["Òy", "Oy", "R"], "Vậy": ["V", "Z"], "Mới": ["Ms"], "Vẫn": ["Vx"], "Yếu": ["Íu"],
           "yếu": ["íu"], "Chiều": ["chìu"], "chiều": ["chìu"], "Tiêu": ["Tiu"], "tiêu": ["tiu"], "Tuổi": ["Tủi"], "tuổi": ["tủi"]}

teen = {"ch": "ck", "ph": "f", "th": "tk", "nh": "nk", "v": "d", "b": "p", "Ch": "Ck", "Ph": "F", "Th": "Tk", "Nh": "Nk", "V": "D", "B": "P","d": "z", "D":"Z"}



telex_dict = {"ă":"aw","â":"aa","á":"as","à":"af","ả":"ar","ã":"ax","ạ":"aj","ắ":"aws","ổ":"oor","ỗ":"oox","ộ":"ooj","ơ":"ow",
"ằ":"awf","ẳ":"awr","ẵ":"awx","ặ":"awj","ó":"os","ò":"of","ỏ":"or","õ":"ox","ọ":"oj","ô":"oo","ố":"oos","ồ":"oof",
"ớ":"ows","ờ":"owf","ở":"owr","ỡ":"owx","ợ":"owj","é":"es","è":"ef","ẻ":"er","ẽ":"ex","ẹ":"ej","ê":"ee","ế":"ees","ề":"eef",
"ể":"eer","ễ":"eex","ệ":"eej","ú":"us","ù":"uf","ủ":"ur","ũ":"ux","ụ":"uj","ư":"uw","ứ":"uws","ừ":"uwf","ử":"uwr","ữ":"uwx",
"ự":"uwj","í":"is","ì":"if","ỉ":"ir","ị":"ij","ĩ":"ix","ý":"ys","ỳ":"yf","ỷ":"yr","ỵ":"yj","đ":"dd",
"Ă":"Aw","Â":"Aa","Á":"As","À":"Af","Ả":"Ar","Ã":"Ax","Ạ":"Aj","Ắ":"Aws","Ổ":"Oor","Ỗ":"Oox","Ộ":"Ooj","Ơ":"Ow",
"Ằ":"AWF","Ẳ":"Awr","Ẵ":"Awx","Ặ":"Awj","Ó":"Os","Ò":"Of","Ỏ":"Or","Õ":"Ox","Ọ":"Oj","Ô":"Oo","Ố":"Oos","Ồ":"Oof",
"Ớ":"Ows","Ờ":"Owf","Ở":"Owr","Ỡ":"Owx","Ợ":"Owj","É":"Es","È":"Ef","Ẻ":"Er","Ẽ":"Ex","Ẹ":"Ej","Ê":"Ee","Ế":"Ees","Ề":"Eef",
"Ể":"Eer","Ễ":"Eex","Ệ":"Eej","Ú":"Us","Ù":"Uf","Ủ":"Ur","Ũ":"Ux","Ụ":"Uj","Ư":"Uw","Ứ":"Uws","Ừ":"Uwf","Ử":"Uwr","Ữ":"Uwx",
"Ự":"Uwj","Í":"Is","Ì":"If","Ỉ":"Ir","Ị":"Ij","Ĩ":"Ix","Ý":"Ys","Ỳ":"Yf","Ỷ":"Yr","Ỵ":"Yj","Đ":"Dd"}

vowel_variants = {
    'a': ['à', 'á', 'ã', 'ạ'],
    'à': ['a', 'á', 'ã', 'ạ'],
    'á': ['à', 'a', 'ã', 'ạ'],
    'ã': ['à', 'á', 'a', 'ạ'],
    'ạ': ['à', 'á', 'ã', 'a'],
    'ă': ['ắ', 'ằ', 'ẵ', 'ặ'],
    'ắ': ['ă', 'ằ', 'ẵ', 'ặ'],
    'ằ': ['ă', 'ắ', 'ẵ', 'ặ'],
    'ẵ': ['ă', 'ắ', 'ằ', 'ặ'],
    'ặ': ['ă', 'ắ', 'ằ', 'ẵ'],
    'â': ['ấ', 'ầ', 'ẫ', 'ậ', 'ẩ'],
    'ấ': ['â', 'ầ', 'ẫ', 'ậ', 'ẩ'],
    'ầ': ['â', 'ấ', 'ẫ', 'ậ', 'ẩ'],
    'ẫ': ['â', 'ấ', 'ầ', 'ậ', 'ẩ'],
    'ậ': ['â', 'ấ', 'ầ', 'ẫ', 'ẩ'],
    'ẩ': ['â', 'ấ', 'ầ', 'ẫ', 'ậ'],
    'e': ['è', 'é', 'ẽ', 'ẹ'],
    'è': ['e', 'é', 'ẽ', 'ẹ'],
    'é': ['è', 'e', 'ẽ', 'ẹ'],
    'ẽ': ['è', 'e', 'é', 'ẹ'],
    'ẹ': ['è', 'e', 'é', 'ẽ'],
    'ê': ['ề', 'ế', 'ễ', 'ệ', 'ể'],
    'ề': ['ê', 'ế', 'ễ', 'ệ', 'ể'],
    'ế': ['ê', 'ê', 'ễ', 'ệ', 'ể'],
    'ễ': ['ê', 'ê', 'ễ', 'ệ', 'ể'],
    'ệ': ['ê', 'ê', 'ễ', 'ệ', 'ể'],
    'ể': ['ê', 'ê', 'ễ', 'ệ', 'ể'],
    'i': ['ì', 'í', 'ĩ', 'ị'],
    'ì': ['i', 'í', 'ĩ', 'ị'],
    'í': ['ì', 'i', 'ĩ', 'ị'],
    'ĩ': ['ì', 'i', 'í', 'ị'],
    'ị': ['ì', 'i', 'í', 'ĩ'],
    'o': ['ò', 'ó', 'õ', 'ọ'],
    'ò': ['o', 'ó', 'õ', 'ọ'],
    'ó': ['o', 'ò', 'õ', 'ọ'],
    'õ': ['o', 'ò', 'ó', 'ọ'],
    'ọ': ['o', 'ò', 'ó', 'õ'],
    'ô': ['ồ', 'ố', 'ỗ', 'ộ', 'ổ'],
    'ồ': ['ô', 'ố', 'ỗ', 'ộ', 'ổ'],
    'ố': ['ô', 'ô', 'ỗ', 'ộ', 'ổ'],
    'ỗ': ['ô', 'ô', 'ồ', 'ộ', 'ổ'],
    'ộ': ['ô', 'ô', 'ồ', 'ố', 'ổ'],
    'ổ': ['ô', 'ô', 'ồ', 'ố', 'ỗ'],
    'ơ': ['ờ', 'ớ', 'ỡ', 'ợ', 'ở'],
    'ờ': ['ơ', 'ớ', 'ỡ', 'ợ', 'ở'],
    'ớ': ['ơ', 'ơ', 'ỡ', 'ợ', 'ở'],
    'ỡ': ['ơ', 'ơ', 'ờ', 'ợ', 'ở'],
    'ợ': ['ơ', 'ơ', 'ờ', 'ớ', 'ở'],
    'ở': ['ơ', 'ơ', 'ờ', 'ớ', 'ỡ'],
    'u': ['ù', 'ú', 'ũ', 'ụ'],
    'ù': ['u', 'ú', 'ũ', 'ụ'],
    'ú': ['u', 'ù', 'ũ', 'ụ'],
    'ũ': ['u', 'ù', 'ú', 'ụ'],
    'ụ': ['u', 'ù', 'ú', 'ũ'],
    'ư': ['ừ', 'ứ', 'ữ', 'ự', 'ử'],
    'ừ': ['ư', 'ứ', 'ữ', 'ự', 'ử'],
    'ứ': ['ư', 'ư', 'ữ', 'ự', 'ử'],
    'ữ': ['ư', 'ư', 'ừ', 'ự', 'ử'],
    'ự': ['ư', 'ư', 'ừ', 'ứ', 'ử'],
    'ử': ['ư', 'ư', 'ừ', 'ứ', 'ữ'],
    'y': ['ỳ', 'ý', 'ỹ', 'ỵ'],
    'ỳ': ['y', 'ý', 'ỹ', 'ỵ'],
    'ý': ['y', 'ỳ', 'ỹ', 'ỵ'],
    'ỹ': ['y', 'ỳ', 'ý', 'ỵ'],
    'ỵ': ['y', 'ỳ', 'ý', 'ỹ']
}
region={"s":"x","l":"n","n":"l","x":"s","d":"gi","S":"X","L":"N","N":"L","X":"S","Gi":"D","D":"Gi"}
global_random = random.uniform(0, 1)
ERROR_TYPES = ["mot", "mot_2", "hai", "ba", "bon", "nam", "bay", "tam", "chin"]
import random
random.seed(42)
def text_to_telex(input_text):
    result = ""
    for char in input_text:
        if char in telex_dict:
            result += telex_dict[char]
        else:
            result += char
    return result

def create_spelling_error(telex_text, error_rate=None, error_type="position"):
    telex_list = list(telex_text)

    if error_rate is None:
        error_rate = random.uniform(0, 1)

    if error_type == "position":
        for i in range(len(telex_list) - 1):
            if telex_list[i] != ' ' and telex_list[i + 1] != ' ':
                if random.random() < error_rate:
                    telex_list[i], telex_list[i + 1] = telex_list[i + 1], telex_list[i]
    elif error_type == "keyboard":
        adjacent_keys = {
            'a': ['q', 'w', 's', 'z'],
            'b': ['v', 'g', 'h', 'n'],
            'c': ['x', 'd', 'f', 'v'],
            'd': ['s', 'e', 'r', 'f', 'c', 'x'],
            'e': ['w', 's', 'd', 'r'],
            'f': ['d', 'r', 't', 'g', 'v', 'c'],
            'g': ['f', 't', 'y', 'h', 'b', 'v'],
            'h': ['g', 'y', 'u', 'j', 'n', 'b'],
            'i': ['u', 'h', 'j', 'k', 'o'],
            'j': ['h', 'u', 'i', 'k', 'm', 'n'],
            'k': ['j', 'i', 'o', 'l', 'm'],
            'l': ['k', 'o', 'p'],
            'm': ['n', 'j', 'k'],
            'n': ['b', 'h', 'j', 'm'],
            'o': ['i', 'k', 'l', 'p'],
            'p': ['o', 'l'],
            'q': ['a', 'w', '1', '2'],
            'r': ['e', 'd', 'f', 't', '4','5'],
            's': ['a', 'w', 'e', 'd', 'f', 'x', 'z'],
            't': ['r', 'f', 'g', 'y'],
            'u': ['y', 'h', 'j', 'i'],
            'v': ['c', 'f', 'g', 'b'],
            'w': ['q', 'a', 's', 'e', 'd'],
            'x': ['z', 's', 'd', 'c'],
            'y': ['t', 'g', 'h', 'u'],
            'z': ['a', 's', 'x']
        }

        for i in range(len(telex_list)):
            if telex_list[i] != ' ':
                if random.random() < error_rate:
                    original_char = telex_list[i]
                    nearby_chars = adjacent_keys.get(original_char, [])
                    if nearby_chars:
                        replacement_char = random.choice(nearby_chars)
                        telex_list[i] = replacement_char

    misspelled_telex = "".join(telex_list)
    return misspelled_telex

def teencode(sentence, error_rate=None):
    words = sentence.split()

    if error_rate is None:
        error_rate = random.uniform(0, 1)

    num_errors = int(error_rate * len(words))

    for _ in range(num_errors):
        word_index = random.randint(0, len(words) - 1)
        word = words[word_index]

        if word == "bạn":
            replace_with = random.choice(["b", "bn", "pạn"])
        elif word in acronym:
            replace_with = random.choice(acronym[word])
        else:
            replace_with = word  # Replace with the word itself, not a list
        if word == "Bạn":
            replace_with = random.choice(["B", "Bn", "Pạn"])
        elif word in acronym:
            replace_with = random.choice(acronym[word])
        else:
            replace_with = word  # Replace with the word itself, not a list
        words[word_index] = replace_with

    # Apply teen slang substitutions after all other replacements
    for key, value in teen.items():
        words = [w.replace(key, value) for w in words]

    # Check for "pn" and replace it with "bn"
    for i, word in enumerate(words):
        if word == "pn":
            words[i] = "bn"
        elif word == "p":
            words[i] = "b"
    for i, word in enumerate(words):
        if word == "Pn":
            words[i] = "Bn"
        elif word == "P":
            words[i] = "B"
    return ' '.join(words)

def tao_loi_doi_vi_tri_tu_trong_cau(input_text, error_rate=None):
    cac_tu = input_text.split()
    tu_loi_doi_vi_tri = list(cac_tu)

    if error_rate is None:
        error_rate = random.uniform(0, 1)

    cac_vi_tri = list(range(len(cac_tu)))
    so_tu_loi = int(len(cac_tu) * error_rate)
    vi_tri_loi = random.sample(cac_vi_tri, so_tu_loi)

    for i in vi_tri_loi:
        j = random.choice([x for x in cac_vi_tri if x != i])
        tu_loi_doi_vi_tri[i], tu_loi_doi_vi_tri[j] = tu_loi_doi_vi_tri[j], tu_loi_doi_vi_tri[i]

    cu_loi_doi_vi_tri = ' '.join(tu_loi_doi_vi_tri)
    return cu_loi_doi_vi_tri

def tao_loi_lap_lai_tu_trong_cau(input_text, error_rate=None):
    # Tách câu thành các từ riêng lẻ
    cac_tu = input_text.split()

    # Tạo danh sách các từ bị lặp lại
    tu_loi_lap_lai = list(cac_tu)

    # Lặp lại mỗi từ trong câu
    for i in range(len(cac_tu)):
        if random.random() < 0.2:  # Sử dụng tỷ lệ lỗi 0.2 trong ví dụ này
            tu_loi_lap_lai[i] = tu_loi_lap_lai[i] + " " + tu_loi_lap_lai[i]

    # Ghép lại các từ để tạo câu mới
    cu_loi_lap_lai = ' '.join(tu_loi_lap_lai)

    return cu_loi_lap_lai

def tao_loi_thay_doi_thanh_trong_tu(input_text, error_rate=None):
    cac_tu = input_text.split()

    # Tạo danh sách các từ với thanh đau bị thay đổi
    tu_loi_thay_doi_thanh = []

    if error_rate is None:
        error_rate = random.uniform(0, 1)

    for tu in cac_tu:
        tu_moi = tu
        if random.random() < error_rate:
            if tu_moi[-1] not in vowel_variants:
                for nguyen_am, bien_te in vowel_variants.items():
                    for bien_te_moi in bien_te:
                        tu_moi = tu_moi.replace(nguyen_am, bien_te_moi)
        tu_loi_thay_doi_thanh.append(tu_moi)

    cu_loi_thay_doi_thanh = ' '.join(tu_loi_thay_doi_thanh)

    return cu_loi_thay_doi_thanh

def tao_loi_xoa_ky_tu_trong_tu(input_text, error_rate=None):
    cac_tu = input_text.split()
    tu_loi_xoa_ky_tu = []

    if error_rate is None:
        error_rate = random.uniform(0, 1)

    for tu in cac_tu:
        tu_moi = tu
        if random.random() < error_rate:
            so_ky_tu_loi = max(1, int(len(tu_moi) * error_rate))
            for _ in range(so_ky_tu_loi):
                vi_tri_loi = random.randint(0, len(tu_moi) - 1)
                tu_moi = tu_moi[:vi_tri_loi] + tu_moi[vi_tri_loi + 1:]
        tu_loi_xoa_ky_tu.append(tu_moi)

    cu_loi_xoa_ky_tu = ' '.join(tu_loi_xoa_ky_tu)

    return cu_loi_xoa_ky_tu

def convert_and_remove_space(sentence, error_rate=None):
    words = sentence.split()
    
    # Randomly select a pair of adjacent words
    if len(words) >= 2 and np.random.uniform(0, 1) < error_rate:
        random_index = np.random.choice(len(words) - 1)
        word_pair = words[random_index:random_index + 2]
        
        # Convert the selected word pair to Telex format
        telex_pair = [text_to_telex(word) for word in word_pair]
        
        # Join the Telex-formatted words and remove the space between them
        sentence = sentence.replace(' '.join(word_pair), ''.join(telex_pair))
    
    return sentence

def region_loi(text, region, error_prob=None):
    result = ""

    if error_prob is None:
        error_prob = random.uniform(0, 1)

    for char in text:
        if random.random() < error_prob and char in region:
            result += region[char]
        else:
            result += char

    return result



In [16]:
def kiem_tra_tu(tu):
    for i in range(len(tu) - 2):
        if tu[i] == 'E' and tu[i+1] == 'N' and tu[i+2] == 'G':
            return True
    return False
def process_file(input_file, output_file):
    with open(input_file, 'r', encoding='utf-8') as file:
        input_text = file.read()

    sentences = input_text.split('.')

    with open(output_file, 'w', encoding='utf-8') as file:
        for sentence in sentences:
            if sentence.strip() != '':
                words = sentence.split()
                for word in words:
                    # Kiểm tra xem từ có phải là ENG không
                    if not kiem_tra_tu(word):
                        error_rate = random.uniform(0, 1)
                        error_type = random.choice(ERROR_TYPES)

                        processed_word = apply_error(word, error_type, error_rate)
                        file.write(processed_word + ' ')
                    else:
                        # Nếu từ là ENG, giữ nguyên từ
                        file.write(word + ' ')
                file.write('\n')

def apply_error(sentence, error_type, error_rate):
    if error_type == "mot":
        return create_spelling_error(sentence, error_rate, "position")
    elif error_type == "mot_2":
        return create_spelling_error(sentence, error_rate, "keyboard")
    elif error_type == "hai":
        return teencode(sentence, error_rate)
    elif error_type == "ba":
        return tao_loi_doi_vi_tri_tu_trong_cau(sentence, error_rate)
    elif error_type == "bon":
        return tao_loi_lap_lai_tu_trong_cau(sentence, error_rate)
    elif error_type == "nam":
        return tao_loi_thay_doi_thanh_trong_tu(sentence, error_rate)
    elif error_type == "bay":
        return tao_loi_xoa_ky_tu_trong_tu(sentence, error_rate)
    elif error_type == "tam":
        return convert_and_remove_space(sentence, error_rate)
    elif error_type == "chin":
        return region_loi(sentence, region, error_rate)

if __name__ == "__main__":
    input_file_path = "test.txt"
    output_file_path = "test_1.txt"
    process_file(input_file_path, output_file_path)